In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import math

import esdglider.gcp as gcp
import esdglider.glider as glider
import esdglider.utils as utils

deployment_info = {
    "deployment": 'calanus-20241019', 
    "project": "ECOSWIM", 
    "mode": 'delayed', 
    "min_dt": '2024-10-19 17:37:00', 
}
deployment = deployment_info["deployment"]
project = deployment_info["project"]
mode = deployment_info["mode"]

year = utils.year_path(project, deployment)
bucket_name = 'amlr-gliders-deployments-dev'
deployments_path = f"/home/sam_woodman_noaa_gov/{bucket_name}"
config_path = f"/home/sam_woodman_noaa_gov/glider-lab/deployment-config"

gcp.gcs_mount_bucket(
    "amlr-gliders-deployments-dev", deployments_path, 
    ro=False)


paths = glider.get_path_deployment(
    deployment_info, deployments_path, config_path)

dir_ts = paths["tsdir"]
path_raw = os.path.join(dir_ts, f"{deployment}-{mode}-raw.nc")
path_sci = os.path.join(dir_ts, f"{deployment}-{mode}-sci.nc")
path_eng = os.path.join(dir_ts, f"{deployment}-{mode}-eng.nc")

dir_ngdac = paths["profdir"]
if os.path.isdir(dir_ngdac):
    files_ngdac = os.listdir(dir_ngdac)
# files_ngdac
# os.path.join(paths["profdir"], f"{deployment} -{mode}.nc")
# path_sci = os.path.join(dir_ts, 'calanus-20241019-sci.nc')
# path_eng = os.path.join(dir_ts, 'calanus-20241019-eng.nc')


In [ ]:
ds_sci = xr.load_dataset(path_sci)
ds_sci

In [ ]:
ds_eng = xr.load_dataset(path_eng)
ds_eng

In [ ]:
import folium
import numpy as np
import xarray as xr
import random
from branca.colormap import linear

# # Example dataset selection from xarray (replace with your own dataset)
# ds = ds_eng #.sel(time=slice("2024-10-19", "2024-10-20"))
# print(f"Number of points: {len(ds.time.values)}")

def folium_play(ds, sample_rate=10):
    # Extract lat, lon, and info (datetime)
    lat = ds["latitude"].values
    lon = ds["longitude"].values
    info = np.datetime_as_string(ds["time"].values, unit='s')
    dates = ds["time"].values  # The datetime values

    # Convert datetime to numerical values (e.g., Unix timestamp or days since a reference date)
    start_date = np.datetime64(ds["time"].values.min())
    days_since_start = (dates - start_date).astype(int)  # Convert timedelta to integer days

    # Create a base map, centered around the average lat/lon
    m = folium.Map(
        location=[np.mean(lat), np.mean(lon)], 
        zoom_start=10, 
        tiles="cartodb positron"
    )

    # Define a color map for the date range
    colormap = linear.YlGnBu_09.scale(min(days_since_start), max(days_since_start))  # Blue scale

    # Downsampling the data for performance improvement (optional)
    downsampled_indices = random.sample(range(len(lat)), len(lat) // sample_rate)  # Adjust sampling rate
    lat_downsampled = lat[downsampled_indices]
    lon_downsampled = lon[downsampled_indices]
    info_downsampled = info[downsampled_indices]
    days_since_start_downsampled = days_since_start[downsampled_indices]

    print(len(lon_downsampled))

    # Add CircleMarkers to the map, colored by the date
    for i in range(len(lat_downsampled)):
        # Map the datetime to a color
        color = colormap(days_since_start_downsampled[i])

        # Create a Tooltip with minimal information
        tooltip = info_downsampled[i]  # Tooltip will show the date

        # Create a CircleMarker with the calculated color
        folium.CircleMarker(
            location=[lat_downsampled[i], lon_downsampled[i]], 
            radius=3,  # Adjust the radius for better visibility
            color=color, 
            fill=True, 
            fill_color=color,  # Use the same color for filling
            fill_opacity=0.6,
            tooltip=tooltip  # Use Tooltip instead of Popup
        ).add_to(m)

    # Add the colormap to the map for reference
    colormap.add_to(m)

    # Display the map
    return m


In [ ]:
ds_raw = xr.load_dataset(path_raw)
ll_good = (
        (ds_raw.longitude >= -180)
        & (ds_raw.longitude <= 180)
        & (ds_raw.latitude >= -90)
        & (ds_raw.latitude <= 90))
ds = ds_raw.where(ll_good, drop = True)
mr = folium_play(ds)
display(mr)

In [ ]:
me = folium_play(ds_eng)
display(me)

In [ ]:
ms = folium_play(ds_sci)
display(ms)

In [ ]:
# import xarray as xr
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature

# # Open the dataset (replace 'your_dataset.nc' with the path to your file)
# # ds = xr.open_dataset('your_dataset.nc')

# # Extract the latitude and longitude values
# lat = ds_sci['latitude'].values  # Assuming 'lat' is the latitude dimension
# lon = ds_sci['longitude'].values  # Assuming 'lon' is the longitude dimension

# # Extract the data values you want to plot (let's assume it is called 'data')
# data = ds_sci['temperature'].values  # This could be your variable of interest

# # Set up the plot with Cartopy projection
# fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})

# # Plot the points on the map
# scatter = ax.scatter(lon, lat, c=data, cmap='viridis', edgecolor='k', marker='o', s=20, transform=ccrs.PlateCarree())

# # Add Cartopy features for a nice map appearance
# ax.coastlines(resolution='10m')  # Draw coastlines with a medium resolution
# ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=2)  # Add country borders
# ax.add_feature(cfeature.LAND, edgecolor='black')  # Add land with black edges
# ax.add_feature(cfeature.LAND, facecolor='lightgray')  # Land color
# # ax.add_feature(cfeature.LAKE, facecolor='lightblue')  # Lakes in light blue
# # ax.add_feature(cfeature.RIVERS)  # Add rivers

# # Add gridlines for latitude and longitude
# ax.gridlines(draw_labels=True, linewidth=1, color='gray', linestyle='--')

# # Add a colorbar
# cbar = plt.colorbar(scatter, ax=ax, orientation='vertical')
# cbar.set_label('Data Value')

# # Set a title
# ax.set_title('Data Points on Map', fontsize=16)

# # Show the plot
# plt.show()


In [ ]:
ds_sci.distance_over_ground.values

In [ ]:
ds_diff = ds_sci.distance_over_ground.values - np.roll(ds_sci.distance_over_ground.values, 1)
# y_diff = y.distance_over_ground.values - np.roll(y.distance_over_ground.values, 1)
display(pd.DataFrame(ds_diff).describe())

np.argmax(ds_diff)

In [ ]:
# ds_sci_grid5 = xr.load_dataset(os.path.join(paths["griddir"], f"{deployment}_grid-{mode}-5m.nc"))
# ds_sci_grid5

In [ ]:
# ds_prof0 = xr.load_dataset(os.path.join(dir_ngdac, files_ngdac[0]))
# display(ds_prof0)

# ds_prof20 = xr.load_dataset(os.path.join(dir_ngdac, files_ngdac[20]))
# display(ds_prof20)

In [ ]:
pd_eng = ds_eng.to_pandas()
pd_sci = ds_sci.to_pandas()

## Unifying profiles

In [ ]:
# x = pd_eng.reset_index()
# # x.profile_index = np.floor(x.profile_index)
# y = x.groupby('profile_index').agg(
#     time_start = ('time', 'min'), 
#     time_end = ('time', 'max'), 
#     depth_first = ('depth', 'first'), 
#     depth_last = ('depth', 'last')
# )
# # y.loc[:-1, "time_end"] = y['time_start'].shift(-1).iloc[:-1]

# y

In [ ]:
# y2 = x.groupby('profile_index').agg(
#     time_start = ('time', 'min')
# ).reset_index()
# print(y2)

# z = pd_sci.drop('profile_index', axis=1)
# d = pd.merge_asof(z, y2, left_index=True, right_on="time_start", direction = "backward")
# d[['depth', 'profile_index', 'profile_direction', 'time_start']]

In [ ]:
# ds_eng2 = utils.get_profiles_esd(ds_eng, "depth")
# pd_eng2 = ds_eng2.to_pandas()

# prof_idx, prof_dir = utils.findProfiles(
#     ds_eng2.time.values, ds_eng2.depth.values, stall=20, shake=200)
# pd_eng2['prof_idx'] = prof_idx
# pd_eng2['prof_dir'] = prof_dir

# # 'latitude', 'longitude', 
# pd_eng2[['depth', 'profile_index',
#         'profile_direction', 'prof_idx', 'prof_dir']]
